<a href="https://colab.research.google.com/github/balaji36918/2D-Image-Processing/blob/main/Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Task 1</h2>

1. Explain the difference between motion filed and optical flow with example.



> While the motion field is the projection from 3D into the 2D picture, the optical flow is bascially the chance of the brightness inside a picture. The change in brightness can be a motion, but the problem is that even a non moving object can look changed while only the origin or number of the light sources was changed. By this the optical flow would show a change while there was no actual movement.


> A typical example for this case is the sign in front of a barber shop. While the actual motion is in the horizontal, it looks if we only look on the brightness that the movement is actually taking a vertical way. Therefore it would be the case that the motion field would be showing a change in the horizontal way, while the optical flow would suggest a move in the vertical way.



--- 
<br>

2. Explain if the optical flow can be used for tracking bubbles in a glass of carbonated water. Discuss it with respect to the three assumptions of optical flow tracking.



> Let us take a look at water bubbles inside the glass in this video:<br> [Carbonated water bubbles in side glass](https://www.youtube.com/watch?v=7ZY9ZyMsizo&t=586s)
<br>Now let us come to our 3 assumptions for tracking optical flow.
<br> <b>1. Brightness Consistency-</b> We can observe that the brightness of the bubbles is consistent, implying that the projection of the same spot appears the same in each frame. So, the first assumption is satisfied.
<br> <b>2. Spatial Coherence-</b> For small patch, one specific pixel in the bubble that we want to track and surrounding pixels have the same motion. Hence the second assumption is satisfied.
<br> <b>3. Temporal Persistence-</b> This assumption states that there shall not be abrupt motion of camera or the object we want to track, rather it shall be gradually updated over time. This is where the optical flow tracking gets tricky because few bubbles move rapidly and change their position very quickly between two frames. Hence this third assumption gets violated for few bubbles but for some bubbles it is satisfied.
<br> <b>Conclusion-</b> From this we can conclude that optical flow for tracking the bubbles in a glass of carbonated water as all assumptions are satisfied with exception that it might be difficult to track fast moving bubbles.




---




<br>

3. Explain the aperture problem in optical flow. How can one solve this problem by estimating the global geometric structure of the scene?



> <br> The aperture problem refers to the fact that the motion of a one-dimensional spatial structure, such as a bar or edge, cannot be determined unambiguously if it is viewed through a small aperture such that the ends of the stimulus are not visible.
<br>Due to aperture problms, similar frames are identified as non similar frames and vice versa.



> The following 2 things are done  to solve the aperture problem through Horn Schunck method to compute optical flow:<br>1) Increasing brightness constancy.<br>
2) Enforcing smooth flow field .






---



4. The proposed Lucas-Kanade method has an issue with large movement. Why? How can you deal with large movement in the image?



> Proposed Lucas-Kanade method has issues with larger movement due to window size. Small window size might miss larger movements and larger window size might be slower. <br>Larger movements can be dealt with coarse to fine registration with iterative refinement.





---



---




<br>

<h2>Task 2</h2>

Analysis of Result:

> The optical flow can be seen as the distribution of apparent velocities of movement of brightness pattern in an image. <br>The flow fields may be used to analyze produce segmentations into regionsthat are associated with moving objects.<br>If we increase the window size in lucas cande parameters, then number of points that we want to detect increases.
If we increase the quality level in ShiTomasi corner detection number of points to be detected decreases.<br>For subsequent runs, equal number of points are not detected as well as same points are not detected, so the results are inconsistent.







In [ ]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture(0)
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

error: ignored